In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import pickle

boundary = pickle.load(open('boundary/boundary.p','rb'))
dem = rasterio.open('dem/aster_merged_3338.tif')
Z_dem = dem.read().squeeze()

x_dem = np.linspace(dem.bounds.left,dem.bounds.right,dem.width)
y_dem = np.linspace(dem.bounds.top,dem.bounds.bottom,dem.height)
X_dem,Y_dem = np.meshgrid(x_dem,y_dem)

x_inds = (x_dem > boundary[:,0].min()-10000) & (x_dem<boundary[:,0].max()+10000)
y_inds = (y_dem > boundary[:,1].min()-10000) & (y_dem<boundary[:,1].max()+40000)
x_dem = x_dem[x_inds]
y_dem = y_dem[y_inds]
Z_dem = Z_dem[y_inds,:][:,x_inds]
skip = 8
x_dem = x_dem[::skip]
y_dem = y_dem[::skip]
Z_dem = Z_dem[::skip,::skip]


ls = LightSource(azdeg=315, altdeg=45)
Z_hillshade = ls.hillshade(Z_dem,vert_exag=3.0,dx=x_dem[1]-x_dem[0],dy=y_dem[1]-y_dem[0])


In [ ]:
EPS = np.finfo(float).eps

def compute_orographic_precip(elevation, dx, dy, pad=200, **param):
    """Compute orographic precipitation.
    Parameters
    ----------
    elevation : array_like
        2D input array of a given elevation
    dx, dy : int
        Horizontal and vertical resolution in [m]
    **param
        A dictionary used to store relevant parameters for computation.
    param kwargs
    ----------------
    latitude (float) : Coriolis effect decreases as latitude decreases
    precip_base (float) : non-orographic, uniform precipitation rate [mm hr-1], usually [0, 10]
    wind_speed (float) : [m s-1]
    wind_dir (float) : wind direction [0: north, 270: west]
    conv_time (float) : cloud water to hydrometeor conversion time [s]
    fall_time (float) : hydrometeor fallout time [s]
    nm (float) : moist stability frequency [s-1]
    hw (float) : water vapor scale height [m]
    cw (float) : uplift sensitivity [kg m-3], product of saturation water vapor sensitivity ref_density [kg m-3] and environmental lapse rate (lapse_rate_m / lapse_rate)
    Returns
    -------
    array_like
        2D array structure the same size as elevation with precipitation rate [mm hr-1]
    """

    # --- wind components
    u0 = -np.sin(param['wind_dir'] * 2 * np.pi / 360) * param['wind_speed']
    v0 = np.cos(param['wind_dir'] * 2 * np.pi / 360) * param['wind_speed']

    # --- other factors
    f_coriolis = 2 * 7.2921e-5 * np.sin(param['latitude'] * np.pi / 180)

    # --- pad raster boundaries prior to FFT
    calc_pad = int(np.ceil(((sum(elevation.shape))) / 2) / 100 * 100)
    #pad = min([calc_pad, pad])

    h = np.pad(elevation, pad, 'constant')
    nx, ny = h.shape

    # --- FFT
    hhat = np.fft.fft2(h)

    x_n_value = np.fft.fftfreq(ny, (1. / ny))
    y_n_value = np.fft.fftfreq(nx, (1. / nx))

    x_len = nx * dx
    y_len = ny * dy
    kx_line = 2 * np.pi * x_n_value / x_len
    ky_line = 2 * np.pi * y_n_value / y_len
    kx = np.tile(kx_line, (nx, 1))
    ky = np.tile(ky_line[:, None], (1, ny))

    # --- vertical wave number (m)
    sigma = kx * u0 + ky * v0

    mf_num = param['nm']**2 - sigma**2
    mf_den = sigma**2 - f_coriolis**2

    # numerical stability
    mf_num[mf_num < 0] = 0.
    mf_den[(mf_den < EPS) & (mf_den >= 0)] = EPS
    mf_den[(mf_den > -EPS) & (mf_den < 0)] = -EPS
    sign = np.where(sigma >= 0, 1, -1)

    m = sign * np.sqrt(np.abs(mf_num / mf_den * (kx**2 + ky**2)))

    # --- transfer function
    P_karot = ((param['cw'] * 1j * sigma * hhat) /
               ((1 - (param['hw'] * m * 1j)) *
                (1 + (sigma * param['conv_time'] * 1j)) *
                (1 + (sigma * param['fall_time'] * 1j))))

    # --- inverse FFT, de-pad, convert units, add uniform rate
    P = np.fft.ifft2(P_karot)
    P = np.real(P[pad:-pad, pad:-pad])
    P *= 3600   # mm hr-1
    P += param['precip_base']
    P[P < 0] = 0

    return P

In [ ]:
dx = x_dem[1] - x_dem[0]
dy = y_dem[1] - y_dem[0]

lapse_rate = -5.8
lapse_rate_m = -6.5
ref_density = 7.4e-3

param = {'latitude': 0,
         'precip_base': 10,  #mm/hr
         'wind_speed': 10,
         'wind_dir': 0,
         'conv_time': 1000,
         'fall_time': 1000,
         'nm': .004,
         'hw': 2000,
         'cw': ref_density * lapse_rate_m / lapse_rate}

P = compute_orographic_precip(Z_dem,dx,dy,pad=200,**param)/1000*24

In [ ]:
#plt.contourf(x_dem,y_dem,P,25)
#plt.colorbar()
plt.imshow(Z_hillshade,extent=(x_dem.min(),x_dem.max(),y_dem.min(),y_dem.max()),cmap=plt.cm.gray)
plt.contourf(x_dem,y_dem,P,10,alpha=0.5,cmap=plt.cm.hot)
plt.colorbar()
plt.plot(*boundary.T,'k-')

line_positions = [720000.,735000]
colors = ['k','r']
for x,c in zip(line_positions,colors):
    plt.axvline(x,color=c)


#plt.contour(x_dem,y_dem,Z_dem,10,colors='black',alpha=0.1)
plt.gcf().set_size_inches(12,12)

In [ ]:
for x,c in zip(line_positions,colors):
    plt.plot(y_dem,Z_dem[:,int(Z_dem.shape[1]*(x-x_dem.min())//(x_dem.max() - x_dem.min()))],c)
plt.ylabel('Elevation (Solid Line)')
plt.twinx()
for x,c in zip(line_positions,colors):
    plt.plot(y_dem,P[:,int(Z_dem.shape[1]*(x-x_dem.min())//(x_dem.max() - x_dem.min()))],c+'--')
plt.ylabel('Precip. m/d (Dashed Line)')

plt.xlabel('Northing')

plt.gcf().set_size_inches(12,4)

In [ ]:
x_dem[1] - x_dem[0]